In [10]:
import pandas as pd
import numpy as np
import math

In [12]:
def clean_text(df):
    for i, t in enumerate(df['Testo']):
        if len(t) < 4:
            df['Testo'][i] = ''
    return df

def fill_null_features(df):
    for c in ['Domanda','Testo']:
        for i in range(0,len(df.index)):  
            if not df[c][i]:
                j=i
                while j>0: 
                    j-=1
                    if df[c][j]:
                        df[c][i] = df[c][j]
                        break
    return df

# Se lo stralcio è vuoto, prendo tutto il testo come stralcio. Non usata su Hyperion
def copy_text(df):
    for i in range(0,len(df.index)):  
            if not df['Stralcio'][i]:
                df['Stralcio'][i] = df['Testo'][i]
    return df

def add_context(df):
    context_list = []
    for i, span in enumerate(df['Testo']):
        
        # first row
        if i == 0:
            context =span + ' ' + df['Stralcio'][i+1]
        #last row
        elif i == len(df.index)-1:
            context = df['Stralcio'][i-1] + ' ' + span
        
        #text with only 1 span
        elif df['Testo'][i] and df['Testo'][i+1]:
            context = span
        #first span
        elif df['Testo'][i]:
            context = span + ' ' + df['Stralcio'][i+1]
        #last span
        elif df['Testo'][i+1]:
            context = df['Stralcio'][i-1] + ' ' + span
        
        else:
            context = df['Stralcio'][i-1] + ' ' + span + ' ' + df['Stralcio'][i+1]
        context_list.append(context)
    df['Contesto']= context_list
    print(len(context_list))
    return df

def uniform_labels(df):
    df['Repertorio'].replace('implicazioni','implicazione', inplace=True)
    df['Repertorio'].replace('previsioni','previsione', inplace=True)



In [13]:
#Hyperion dataset
df = pd.read_csv('../data/Original_csv/Hyperion.csv', na_filter=False)
df = clean_text(df)
df = add_context(df)
df['Repertorio'] = df['Repertorio'].str.lower()
uniform_labels(df)


print('Numero stralci: ' + str(len(df.index)))
filter = df["Repertorio"] != ""
df = df[filter]
#df = df.applymap(str.lower)
print('Numero stralci dopo eliminazione: ' + str(len(df.index)))


35474
Numero stralci: 35474
Numero stralci dopo eliminazione: 35148


In [9]:
df.loc[0]

Domanda       A cosa serve questo testo per il perseguimento...
Testo         Dunque vediamo se ho capito: Conte le canta a ...
Stralcio                           Dunque vediamo se ho capito:
Repertorio                             dichiarazione di intenti
Contesto      Dunque vediamo se ho capito: Conte le canta a ...
Name: 0, dtype: object

In [14]:
df.to_csv ('../data/Original_csv/Hyperion.csv', index = False, header=True )

In [16]:
df.groupby('Repertorio')['Repertorio'].count()

Repertorio
anticipazione                 53
causa                        473
commento                    4403
conferma                     639
considerazione               312
contrapposizione            1413
deresponsabilizzazione       638
descrizione                 4882
dichiarazione di intenti     643
generalizzazione            1066
giudizio                    2764
giustificazione              440
implicazione                 706
non risposta                1163
opinione                    1343
possibilità                  616
prescrizione                1920
previsione                  1170
proposta                     256
ridimensionamento           1102
sancire                     5627
specificazione              1396
valutazione                 2123
Name: Repertorio, dtype: int64

# Split

In [15]:
# split
test_set_perc = 0.2

gb = df.groupby('Repertorio')
train_list = []
test_list = []

for x in gb.groups:
    class_df = gb.get_group(x)

    # Test set creation
    test = class_df.sample(frac=test_set_perc, random_state=1464)
    train = pd.concat([class_df,test]).drop_duplicates(keep=False)

    train_list.append(train)
    test_list.append(test)

train_df = pd.concat(train_list)
test_df = pd.concat(test_list)


train_df.to_csv ('../data/Splitted_full/Hyperion_train.csv', index = False, header=True )
test_df.to_csv ('../data/Splitted_full/Hyperion_test.csv', index = False, header=True )



    

In [17]:
train_df.groupby('Repertorio')['Repertorio'].count()

Repertorio
anticipazione                 42
causa                        378
commento                    3506
conferma                     511
considerazione               250
contrapposizione            1130
deresponsabilizzazione       506
descrizione                 3893
dichiarazione di intenti     513
generalizzazione             851
giudizio                    2207
giustificazione              350
implicazione                 564
non risposta                 930
opinione                    1074
possibilità                  493
prescrizione                1535
previsione                   934
proposta                     205
ridimensionamento            879
sancire                     4497
specificazione              1117
valutazione                 1698
Name: Repertorio, dtype: int64